<a href="https://colab.research.google.com/github/tempdata73/crowd_analysis/blob/master/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSRNet: Congested Scene Recognition Network
The purpose of this notebook is to train a CSRNet with the help of GPU instances.

In [0]:
!pip install PyDrive -qq

In [0]:
!git clone https://github.com/tempdata73/crowd_analysis.git
%cd crowd_analysis

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
FILE_ID = '16dhJn7k4FWVwByRsQAEpl9lwjuV03jVI'
FILENAME = 'shanghai_dataset.zip'
DST_DIR = 'Shanghai'

download = drive.CreateFile({'id': FILE_ID})
download.GetContentFile(FILENAME)

In [0]:
from zipfile import ZipFile

ZipFile(FILENAME).extractall(DST_DIR)

In [0]:
!rm shanghai_dataset.zip

## Density map generation

In [0]:
import time
start_time = time.time()
!python make_dataset.py Shanghai/ Shanghai_A
end_time = time.time()
print('Density map generation took {:0.2f} secs'.format(end_time - start_time))

In [0]:
!python train.py part_A/train_val.json part_A/test.json

## Model tests

In [0]:
import torch
import os

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
CKPT_DIR = 'ckpts'
models = []

for model_fn in os.listdir(CKPT_DIR):
  if model_fn == 'model.pth.tar':
    continue
  model_path = os.path.join(CKPT_DIR, model_fn)
  ckpt = torch.load(model_path, map_location=torch.device(device))
  models.append((model_path, ckpt['loss']))

models = sorted(models, key=lambda loss: loss[1], reverse=False)
model_path = models[0][0]
print(model_path)

In [0]:
!python validation.py $model_path part_A/test.json

In [0]:
import json

with open('data/metrics.json') as infile:
  data = json.load(infile)

data

In [0]:
# Last epoch did not necessarily yield best model
# Fix data according to that

ckpt = torch.load(model_path, map_location=torch.device(device))
last_epoch = ckpt['epoch']

with open('data/loss_data.json') as infile:
  loss = json.load(infile)

last_ckpt_loss_data = {'train_mae': loss['train_mae'][:last_epoch], 'val_mae': loss['val_mae'][:last_epoch]}

with open('data/loss_data.json', 'w') as outfile:
  json.dump(last_ckpt_loss_data, outfile)

In [0]:
# Download model and data
from google.colab import files

files.download(model_path)
files.download('data/metrics.json')
files.download('data/loss_data.json')